In [13]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.layers.core import Flatten, Dense, Dropout
# from keras.layers.normalization import BatchNormalization
from keras import optimizers
from sklearn.utils import shuffle
from keras.models import Model
import time 
import pickle
from keras.callbacks import Callback
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping


In [14]:

# GLOBAL VARIABLES
path = "data1/NITROHCS_V1.0/"
inputShape = (32, 32, 3)
batchsizeVal = 64
epochsVal = 50
# Importing Classes
myList = os.listdir(path)
noofclasses = len(myList)
print("No of Classes Detected =", noofclasses)
images = []
classid = []
# Importing Images
for x in range(0, noofclasses):
    picList = os.listdir(path + "/" + myList[x])
    for y in picList:
        curImg = cv2.imread(path + "/" + myList[x] + "/" + y)
        curImg = cv2.resize(curImg, (inputShape[0], inputShape[1]))
        images.append(curImg)
        classid.append(x)
    print(x, end=" ")
print(" ")
print("No of images imported =", len(images))

# Convert to numpy array
images = np.array(images)
classid = np.array(classid)


No of Classes Detected = 47
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46  
No of images imported = 15040


In [15]:

# Splitting the Data
classid = classid.reshape(-1, 1)
onehotencoder = OneHotEncoder()
Y = onehotencoder.fit_transform(classid).toarray()

images, Y = shuffle(images, Y, random_state=69)

x_train, x_test, y_train, y_test = train_test_split(images, Y, test_size=0.2, random_state=69)
x_train, x_validation, y_train, y_validation = train_test_split(
    x_train, y_train, test_size=0.2, random_state=69
)


In [16]:

# Preprocessing
# def preprocessing(img):
#     img = img / 255
#     # img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#     # img = cv2.equalizeHist(img)
#     return img

# X_train = np.array(list(map(preprocessing, x_train)))
# X_test = np.array(list(map(preprocessing, x_test)))
# X_validation = np.array(list(map(preprocessing, x_validation)))
# Reshape images
X_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 3)
X_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 3)
X_validation = x_validation.reshape(
    x_validation.shape[0], x_validation.shape[1], x_validation.shape[2], 3
)


In [17]:

# Augmenting Images
dataGen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    shear_range=0.1,
    rotation_range=10,
)
dataGen.fit(x_train)
class_weights = {}
unique, counts = np.unique(classid, return_counts=True)
total = np.sum(counts)
for i in range(len(unique)):
    class_weights[unique[i]] = total / (len(unique) * counts[i])
    


In [18]:
# VGG16 model
vgg = VGG16(input_shape=inputShape, weights='imagenet', include_top=False)
# for layer in vgg.layers:
#     layer.trainable = False
for i, layer in enumerate(vgg.layers):
    if i < len(vgg.layers) - 2:
        layer.trainable = False
    else:
        layer.trainable = True

X_train = preprocess_input(X_train) 
X_test = preprocess_input(X_test)
x = Flatten()(vgg.output)
# dense_layer1 = Dense(512, activation='relu')
    #add another fully connected layers with batch norm and dropout
x = Dense(4096, activation='relu')(x)
# x = BatchNormalization()(x)
x = Dropout(0.5)(x)
    
    # #add another fully connected layers with batch norm and dropout
    # x = Dense(4096, activation='relu')(x)
    # x = BatchNormalization()(x)
    # x = Dropout(dropoutRate)(x)
prediction = Dense(noofclasses, activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0   

In [19]:

adam = optimizers.Adam(learning_rate=1e-6)
model.compile(
    loss='categorical_crossentropy',
    optimizer=adam,
    metrics=['accuracy']

)
# Saving the model according to the conditions  
checkpoint = ModelCheckpoint("Resnet_100.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
tensorboard_callback = TensorBoard(log_dir='./logs')
class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)
time_callback = TimeHistory()

# Early Stopping is a regularization method (To minimize overfitting)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')
history = model.fit(dataGen.flow(X_train, y_train, batch_size=batchsizeVal),
                    epochs=epochsVal,
                    validation_data=(X_validation, y_validation),
                    shuffle=True,
                    class_weight=class_weights,
                    verbose=1,callbacks=[tensorboard_callback,time_callback])


Epoch 1/70
151/151 [==============================] - 128s 842ms/step - loss: 4.9152 - accuracy: 0.1368 - val_loss: 2.4880 - val_accuracy: 0.3398
Epoch 2/70
151/151 [==============================] - 118s 782ms/step - loss: 2.6606 - accuracy: 0.3071 - val_loss: 1.9834 - val_accuracy: 0.4479
Epoch 3/70
151/151 [==============================] - 113s 749ms/step - loss: 2.2314 - accuracy: 0.3854 - val_loss: 1.8063 - val_accuracy: 0.4923
Epoch 4/70
151/151 [==============================] - 109s 719ms/step - loss: 1.9274 - accuracy: 0.4554 - val_loss: 1.6785 - val_accuracy: 0.5193
Epoch 5/70
151/151 [==============================] - 117s 778ms/step - loss: 1.7498 - accuracy: 0.4977 - val_loss: 1.5353 - val_accuracy: 0.5746
Epoch 6/70
151/151 [==============================] - 606s 4s/step - loss: 1.5963 - accuracy: 0.5343 - val_loss: 1.7945 - val_accuracy: 0.5538
Epoch 7/70
151/151 [==============================] - 12677s 85s/step - loss: 1.5006 - accuracy: 0.5633 - val_loss: 1.5287 - va

In [ ]:

score = model.evaluate(X_test, y_test, verbose=0)
print('Test Loss Score =', score[0])
print('Test Accuracy =', score[1])

In [ ]:
# Saving the model
pickle_out = open("model_trained_iitbbs_googlenet.p", "wb")
pickle.dump(model, pickle_out)
pickle_out.close()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
y_pred = model.predict(x_test)
y_pred = (y_pred >= 0.5)

print('Classification Report')
print(classification_report(y_test,y_pred))